This is an [R Markdown](http://rmarkdown.rstudio.com) Notebook. When you execute code within the notebook, the results appear beneath the code.

Try executing this chunk by clicking the *Run* button within the chunk or by placing your cursor inside it and pressing *Cmd+Shift+Enter*.

In [2]:
%load_ext rpy2.ipython

In [3]:
%%R

# devtools::install_version('BiocManager', version = '3.10', repos='http://cran.us.r-project.org')
# library(BiocManager)
# BiocManager::install('pd.hugene.2.0.st')
# BiocManager::install('affy')
# BiocManager::install('sva')
# BiocManager::install('limma')
# BiocManager::install('RSQLite')
# BiocManager::install('hrbrthemes')
library(BiocManager)
library(Biobase)
library('hugene21stprobeset.db')
library('hugene21sttranscriptcluster.db')

library('pd.hugene.2.1.st')
# pd.hugene.2.1.st::pd.hugene.2.1.st@tableInfo$tbl
library(oligo)
library(limma)
library(sva)
library(ggplot2)
library(ballgown)

R[write to console]: Bioconductor version 3.16 (BiocManager 1.30.19), R 4.2.2 (2022-10-31)

R[write to console]: Loading required package: BiocGenerics

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, aperm, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


R[write to console]: Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citatio

In [ ]:
%%R
# https://www.bioconductor.org/packages/devel/workflows/vignettes/RNAseq123/inst/doc/limmaWorkflow.html


# Load probeset information
options(timeout = max(300, getOption("timeout")))
options(download.file.method.GEOquery = "wget")
GEOquery::getGEOSuppFiles("GSE119207", baseDir = "data")
# Load the expression data

# data <- oligo::read.celfiles(filenames=affy::list.celfiles(wdir, full.names = TRUE))

In [15]:
%%R
wdir <-"data/GSE119207_RAW"
utils::untar("./data/GSE119207/GSE119207_RAW.tar", exdir="data/GSE119207_RAW")

In [7]:
%%R
wdir <-"data/GSE119207_RAW"
data <- oligo::read.celfiles(filenames=affy::list.celfiles(wdir, full.names = TRUE), pkgname="pd.hugene.2.1.st")


R[write to console]: Platform design info loaded.



Reading in : data/GSE119207_RAW/GSM3360727_MirW1_1_MW_HuGene-2_0-st_.CEL.gz
Reading in : data/GSE119207_RAW/GSM3360728_MirW2_2_MW_HuGene-2_0-st_.CEL.gz
Reading in : data/GSE119207_RAW/GSM3360729_MirW3_3_MW_HuGene-2_0-st_.CEL.gz
Reading in : data/GSE119207_RAW/GSM3360730_MirW4_4_MW_HuGene-2_0-st_.CEL.gz
Reading in : data/GSE119207_RAW/GSM3360731_MirW10_5_L_HuGene-2_0-st_.CEL.gz
Reading in : data/GSE119207_RAW/GSM3360732_MirW15_5_N_HuGene-2_0-st_.CEL.gz
Reading in : data/GSE119207_RAW/GSM3360733_MirW11_6_L_HuGene-2_0-st_.CEL.gz
Reading in : data/GSE119207_RAW/GSM3360734_MirW16_6_N_HuGene-2_0-st_.CEL.gz
Reading in : data/GSE119207_RAW/GSM3360735_MirW12_7_L_HuGene-2_0-st_.CEL.gz
Reading in : data/GSE119207_RAW/GSM3360736_MirW17_7_N_HuGene-2_0-st_.CEL.gz
Reading in : data/GSE119207_RAW/GSM3360737_MirW13_8_L_HuGene-2_0-st_.CEL.gz
Reading in : data/GSE119207_RAW/GSM3360738_MirW18_8_N_HuGene-2_0-st_.CEL.gz
Reading in : data/GSE119207_RAW/GSM3360739_MirW14_9_L_HuGene-2_0-st_.CEL.gz
Reading in :

In [10]:
%%R
data

GeneFeatureSet (storageMode: lockedEnvironment)
assayData: 2598544 features, 18 samples 
  element names: exprs 
protocolData
  rowNames: GSM3360727_MirW1_1_MW_HuGene-2_0-st_.CEL.gz
    GSM3360728_MirW2_2_MW_HuGene-2_0-st_.CEL.gz ...
    GSM3360744_MirW9_18_MW_HuGene-2_0-st_.CEL.gz (18 total)
  varLabels: exprs dates
  varMetadata: labelDescription channel
phenoData
  rowNames: GSM3360727_MirW1_1_MW_HuGene-2_0-st_.CEL.gz
    GSM3360728_MirW2_2_MW_HuGene-2_0-st_.CEL.gz ...
    GSM3360744_MirW9_18_MW_HuGene-2_0-st_.CEL.gz (18 total)
  varLabels: index
  varMetadata: labelDescription channel
featureData: none
experimentData: use 'experimentData(object)'
Annotation: pd.hugene.2.1.st 


In [12]:
%%R
sampleNames(data)

 [1] "GSM3360727_MirW1_1_MW_HuGene-2_0-st_.CEL.gz" 
 [2] "GSM3360728_MirW2_2_MW_HuGene-2_0-st_.CEL.gz" 
 [3] "GSM3360729_MirW3_3_MW_HuGene-2_0-st_.CEL.gz" 
 [4] "GSM3360730_MirW4_4_MW_HuGene-2_0-st_.CEL.gz" 
 [5] "GSM3360731_MirW10_5_L_HuGene-2_0-st_.CEL.gz" 
 [6] "GSM3360732_MirW15_5_N_HuGene-2_0-st_.CEL.gz" 
 [7] "GSM3360733_MirW11_6_L_HuGene-2_0-st_.CEL.gz" 
 [8] "GSM3360734_MirW16_6_N_HuGene-2_0-st_.CEL.gz" 
 [9] "GSM3360735_MirW12_7_L_HuGene-2_0-st_.CEL.gz" 
[10] "GSM3360736_MirW17_7_N_HuGene-2_0-st_.CEL.gz" 
[11] "GSM3360737_MirW13_8_L_HuGene-2_0-st_.CEL.gz" 
[12] "GSM3360738_MirW18_8_N_HuGene-2_0-st_.CEL.gz" 
[13] "GSM3360739_MirW14_9_L_HuGene-2_0-st_.CEL.gz" 
[14] "GSM3360740_MirW19_9_N_HuGene-2_0-st_.CEL.gz" 
[15] "GSM3360741_MirW6_15_MW_HuGene-2_0-st_.CEL.gz"
[16] "GSM3360742_MirW7_16_MW_HuGene-2_0-st_.CEL.gz"
[17] "GSM3360743_MirW8_17_MW_HuGene-2_0-st_.CEL.gz"
[18] "GSM3360744_MirW9_18_MW_HuGene-2_0-st_.CEL.gz"


In [20]:
%%R
samples <- sub('.*?_', '', sampleNames(data))
samples <- sub('.*?_', '', samples)
samples <- sub('*_.CEL.gz', '', samples)

sampleNames(data) <- samples

In [19]:
%%R
samples

 [1] "1_MW_HuGene-2_0-st"  "2_MW_HuGene-2_0-st"  "3_MW_HuGene-2_0-st" 
 [4] "4_MW_HuGene-2_0-st"  "5_L_HuGene-2_0-st"   "5_N_HuGene-2_0-st"  
 [7] "6_L_HuGene-2_0-st"   "6_N_HuGene-2_0-st"   "7_L_HuGene-2_0-st"  
[10] "7_N_HuGene-2_0-st"   "8_L_HuGene-2_0-st"   "8_N_HuGene-2_0-st"  
[13] "9_L_HuGene-2_0-st"   "9_N_HuGene-2_0-st"   "15_MW_HuGene-2_0-st"
[16] "16_MW_HuGene-2_0-st" "17_MW_HuGene-2_0-st" "18_MW_HuGene-2_0-st"


normalize probes

In [21]:
%%R

data.rma <- oligo::rma(data)
data.rma

Background correcting
Normalizing
Calculating Expression
ExpressionSet (storageMode: lockedEnvironment)
assayData: 53617 features, 18 samples 
  element names: exprs 
protocolData
  rowNames: 1_MW_HuGene-2_0-st 2_MW_HuGene-2_0-st ...
    18_MW_HuGene-2_0-st (18 total)
  varLabels: exprs dates
  varMetadata: labelDescription channel
phenoData
  rowNames: 1_MW_HuGene-2_0-st 2_MW_HuGene-2_0-st ...
    18_MW_HuGene-2_0-st (18 total)
  varLabels: index
  varMetadata: labelDescription channel
featureData: none
experimentData: use 'experimentData(object)'
Annotation: pd.hugene.2.1.st 


assign sample states

In [22]:
%%R

samples <- sampleNames(data.rma)
samples[grep("1_MW", colnames(data.rma))] <- 0
samples[grep("2_MW", colnames(data.rma))] <- 0
samples[grep("3_MW", colnames(data.rma))] <- 0
samples[grep("4_MW", colnames(data.rma))] <- 0
samples[grep("L", colnames(data.rma))] <- 1
samples[grep("17_MW", colnames(data.rma))] <- 1
samples[grep("15_MW", colnames(data.rma))] <- 1
samples[grep("N", colnames(data.rma))] <- 1
samples[grep("18_MW", colnames(data.rma))] <- 1
samples[grep("16_MW", colnames(data.rma))] <- 1

status <- samples
data.rma@phenoData$status <- samples


# idx 1-4 are healthy after that sort by n and l
samples <- sampleNames(data.rma)
samples[grep("L", colnames(data.rma))] <- "Lesional"
samples[grep("17_MW", colnames(data.rma))] <- "Lesional"
samples[grep("15_MW", colnames(data.rma))] <- "Lesional"
samples[grep("N", colnames(data.rma))] <- "Nonlesional"
samples[grep("18_MW", colnames(data.rma))] <- "Nonlesional"
samples[grep("16_MW", colnames(data.rma))] <- "Nonlesional"
samples[grep("1_MW", colnames(data.rma))] <- "Healthy"
samples[grep("2_MW", colnames(data.rma))] <- "Healthy"
samples[grep("3_MW", colnames(data.rma))] <- "Healthy"
samples[grep("4_MW", colnames(data.rma))] <- "Healthy"
data.rma@phenoData$sampleType <- samples


lesional <- data.rma[,data.rma@phenoData$sampleType == "Lesional"]
h <- data.rma[,data.rma@phenoData$sampleType == "Healthy"]
h_l <- combine(lesional, h)
patients <- c(1,2,3,4,5,5,6,6,7,7,8,8,9,9,10,10,11,11)

In [23]:
%%R

samps <- factor(pData(h_l)$sampleType)
d_mod <- model.matrix(~0 + samps)
colnames(d_mod) <- c('Healthy', "Lesional")
cont <- makeContrasts("Lesional - Healthy", levels=d_mod)

d_fit <- limma::lmFit(exprs(h_l), d_mod)
d_fit <- contrasts.fit(d_fit, cont)
d_bay <- limma::eBayes(d_fit)
data.de <- limma::topTable(d_bay, genelist = , adjust.method = 'BH', sort.by = 'p', number = 10000, coef = 1)

In [25]:
%%R

out <- na.omit(AnnotationDbi::select(hugene21sttranscriptcluster.db,
                             keys = probeNames(data),
                             columns = c('SYMBOL', 'GENENAME', 'ENTREZID'),
                             keytype = "PROBEID"))

data.de$PROBEID <- rownames(data.de)
data.to.plot <-merge(out, data.de, by.y = c("PROBEID"), sort = FALSE)
# aggregate(exprs(data.rma), by = list(rownames(exprs(data.rma))), FUN = mean, na.rm = TRUE)
data.final <- unique(data.to.plot)
data.final <- na.omit(data.final)
data.final


R[write to console]: 'select()' returned many:many mapping between keys and columns

R[write to console]: Error in file(file, ifelse(append, "a", "w")) : 
  cannot open the connection

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In file(file, ifelse(append, "a", "w")) :
R[write to console]: 
 
R[write to console]:  cannot open file '/data/GSE119207_DLE.csv': No such file or directory




Error in file(file, ifelse(append, "a", "w")) : 
  cannot open the connection


RInterpreterError: Failed to parse and evaluate line '\nout <- na.omit(AnnotationDbi::select(hugene21sttranscriptcluster.db,\n                             keys = probeNames(data),\n                             columns = c(\'SYMBOL\', \'GENENAME\', \'ENTREZID\'),\n                             keytype = "PROBEID"))\n\ndata.de$PROBEID <- rownames(data.de)\ndata.to.plot <-merge(out, data.de, by.y = c("PROBEID"), sort = FALSE)\n# aggregate(exprs(data.rma), by = list(rownames(exprs(data.rma))), FUN = mean, na.rm = TRUE)\ndata.final <- unique(data.to.plot)\ndata.final <- na.omit(data.final)\ndata.final\nwrite.csv(data.final, "/data/GSE119207_DLE.csv")\n'.
R error message: 'Error in file(file, ifelse(append, "a", "w")) : \n  cannot open the connection'

In [26]:
%%R

write.csv(data.final, "./data/GSE119207_DLE.csv")

In [ ]:
# rma_pca <- prcomp(t(exprs(data.rma)))
# r_pca <- as.data.frame(rma_pca$x[, 1:2])
# r_pca$color <- pData(data.rma)$sampleType
# batch_pca <- prcomp(t(data.normed_batched))
# b_pca <- as.data.frame(batch_pca$x[, 1:2])
# b_pca$color <- (c(batch1, batch2))
# r = ggplot(r_pca, aes(PC1, PC2, color=color))
# r <- r+geom_point()+ geom_text(aes(label=data.rma@phenoData$sampleType, hjust=0, vjust=0),  size=2)
# b = ggplot(b_pca, aes(PC1, PC2, color=color), label = colnames(data.normed_batched))
# b <- c+geom_point() + geom_text(aes(label=data@phenoData$groups, hjust=0, vjust=0), position = position_dodge(width=0.9),  size=2)
# gridExtra::grid.arrange(r, b, ncol=2)

Add a new chunk by clicking the *Insert Chunk* button on the toolbar or by pressing *Cmd+Option+I*.

When you save the notebook, an HTML file containing the code and output will be saved alongside it (click the *Preview* button or press *Cmd+Shift+K* to preview the HTML file).

The preview shows you a rendered HTML copy of the contents of the editor. Consequently, unlike *Knit*, *Preview* does not run any R code chunks. Instead, the output of the chunk when it was last run in the editor is displayed.